<a href="https://colab.research.google.com/github/Squirtle007/CUDA_Quantum/blob/main/colab/cudaq_tutorial_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set up CUDA Quantum Environment**

In [1]:
#The following commands are used to set up the environment in Colab
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.0-1_all.deb
!dpkg -i cuda-keyring_1.0-1_all.deb
!apt-get update
!apt-get -y install libcublas-11-8 libcusolver-11-8 cuda-cudart-11-8

%pip install cuda-quantum==0.6.0

--2024-04-09 09:45:00--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.0-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4332 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.0-1_all.deb’

cuda-keyring_1.0-1_ 100%[===================>]   4.23K  --.-KB/s    in 0s      

2024-04-09 09:45:00 (261 MB/s) - ‘cuda-keyring_1.0-1_all.deb’ saved [4332/4332]

(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.0-1_all.deb ...
Unpacking cuda-keyring (1.0-1) over (1.0-1) ...
Setting up cuda-keyring (1.0-1) ...

A deprecated public CUDA GPG key appear to be installed.
To remove the key, run this command:
sudo apt-key del 7fa2af80

Get:1 https://cloud.r-project.org/bin/linux/ubuntu ja

# Recap


    
* CUDA Quantum kernels
    https://nvidia.github.io/cuda-quantum/latest/using/cudaq/kernel.html
* Algorithmic primitives
    https://nvidia.github.io/cuda-quantum/latest/index.html
* Targets
    https://github.com/NVIDIA/cuda-quantum

# CUDA Quantum Tutorial #2




* Kernel builder
    https://nvidia.github.io/cuda-quantum/latest/using/cudaq/builder.html

* Simulation backends
    https://nvidia.github.io/cuda-quantum/latest/using/simulators.html

* Variational algorithms
    https://nvidia.github.io/cuda-quantum/latest/using/cudaq/variational.html


Outline

- 1. CUDA Quantum Kernels continued
    - 1.1 Adjoint
    - 1.2 Conditionals

- 2. Variational quantum algorithms (VQAs)
    - 2.1 General
    - 2.2 VQA in CUDA Q
        - 2.2.1 Using cudaq::observe and optimize explicitly
        - 2.2.2 using cudaq.vqe()

- 3. Targets continued
    - 3.1 Hardware backends
        - 3.1.1 Quantum hadrware integration
    - 3.2 Quantum Circuit Simulation
        - 3.2.1 cuQuantum Simulator backends
            - Statevector (custatevec)
            - Tensornet (cutensornet)
        - 3.2.2 Density-matrix (dm)

- 4. Noise modeling
    - 4.1 Background
    - 4.2 With density-matrix (dm) backend

    

### 1. CUDA Quantum Kernels continued

#### 1.1 Adjoint of a Kernel

In [8]:
import cudaq

# Create a kernel and do some operations
other_kernel = cudaq.make_kernel()
other_qubit = other_kernel.qalloc()
other_kernel.x(other_qubit)

# Create a kernel, which'll be the adjoint of other_kernel
kernel = cudaq.make_kernel()
kernel.adjoint(other_kernel)


#### 1.2 Conditional Measurement

In [56]:
# The conditional measurement functionality of `cudaq.kernel`
import cudaq

cudaq.set_target("nvidia")

kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

def then_function():
    kernel.x(qubit)

kernel.x(qubit)

# Measure the qubit.
measurement_ = kernel.mz(qubit)
# Apply `then_function` to the `kernel` if
# the qubit was measured in the 1-state.
kernel.c_if(measurement_, then_function)

# Measure the qubit again.
result = cudaq.sample(kernel, shots_count=30)
print(result)

# Qubit should be in the 0-state after undergoing
# two X rotations.
print("The count of '0' state:", result['0'])

{ 
  __global__ : { 0:30 }
   auto_register_14 : { 1:30 }
}

The count of '0' state: 30


### Variational Algorithms

<center>
<img src="./variational_schematic.png" alt="Variational Schematic" style="width: 65%;">

<img src="./vqa_apps.png" alt="VQA Apps" style="width: 65%;">
</center>

### 2. Variational Algorithms in CUDA Quantum



2.1 Variational algorithms in CUDA Quantum typically leverage the `cudaq.observe(...)` function in tandem with the `cudaq.optimizer`.

One can choose an optimization strategy provided as specific sub-types of the `cudaq.optimizer`.

In [59]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])


# Observable
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 5

def cost_function(x):
    # cudaq.observe() produces the expected value of a specified observable wrt a given parameterized ansatz at given params.
    # This value is the cost function wrt which we are optimizing.
    observeResult = cudaq.observe(kernel, hamiltonian, x)
    print (observeResult.expectation(), x)
    return observeResult.expectation()

# Carry out the optimization
opt_value, opt_theta = optimizer.optimize(dimensions=1, function=cost_function)

-0.999999761581421 [0.0]
5.960464499743523e-08 [1.5707963267948966]
-1.9999997019767757 [-1.5707963267948963]
-0.9999996125698125 [-3.141592653589793]
-0.999999761581421 [2.220446049250313e-16]


#### 2.2 VQE wrapper

In [60]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])

# Hamiltonian operator
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 5

# Carry out the optimization
opt_value, opt_theta = cudaq.vqe(kernel=kernel,
                        spin_operator=hamiltonian,
                        optimizer=optimizer,
                        parameter_count=1)

print(f"\nminimized <H> = {round(opt_value,16)}")
print(f"optimal theta = {round(opt_theta[0],16)}")


minimized <H> = -1.9999997019767757
optimal theta = -1.5707963267948963


### 3. CUDA Quantum `target`
A `target` is a specification of the desired platform and simulator / QPU. It can be specified as a runtime flag in Python. Alteratively, it can also be specified within the application code.

     Simulation backends
    - state-vector (`cuStateVec`)
    - tensor-network (`cuTensorNet`)
    - density-matrix (`dm`)

     Hardware support
    - CPU only, multi-threaded   
    - Single GPU   
    - Multi-GPU
    - Multi-QPU
    - Multi-node
    - QPU

In [61]:
# Print all the available targets on your machine
import cudaq

targets = cudaq.get_targets()

for t in targets:
     print(t)

Target tensornet-mps
	simulator=tensornet_mps
	platform=default
	description=

Target density-matrix-cpu
	simulator=dm
	platform=default
	description=The Density Matrix CPU Target provides a simulated QPU via OpenMP-enabled, CPU-only density matrix emulation.

Target iqm
	simulator=qpp
	platform=default
	description=

Target orca
	simulator=qpp
	platform=default
	description=

Target nvidia-mgpu
	simulator=nvidia_mgpu
	platform=default
	description=

Target nvidia-fp64
	simulator=custatevec_fp64
	platform=default
	description=The NVIDIA FP64 Target provides a simulated QPU via single-GPU cuStateVec integration on FP64 types.

Target nvidia
	simulator=custatevec_fp32
	platform=default
	description=The NVIDIA Target provides a simulated QPU via single-GPU cuStateVec integration on FP32 types.

Target remote-mqpu
	simulator=qpp
	platform=mqpu
	description=

Target nvidia-mqpu
	simulator=custatevec_fp32
	platform=mqpu
	description=The NVIDIA MQPU Target provides a simulated QPU for every a

#### 3.1 Quantum Hardware Integration

In [ ]:
# This code will give an error!!!!!
import cudaq

# Set the target
cudaq.set_target("quantinuum")

# Create the kernel we'd like to execute on Quantinuum.
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)
kernel.h(qubits[0])
kernel.cx(qubits[0], qubits[1])
kernel.mz(qubits)

# Submit to Quantinuum's endpoint and confirm the program is valid.

# By using the synchronous `cudaq.sample`, the execution of
# any remaining classical code in the file will occur only
# after the job has been executed by the Quantinuum service.
# We will use the synchronous call to submit to the syntax
# checker to confirm the validity of the program.
counts = cudaq.sample(kernel)
print(counts)
assert (len(counts) == 2)
assert ('00' in counts)
assert ('11' in counts)

#### 3.2 Quantum Circuit Simulation



What is quantum circuit simulation?

Emulating the behavior of a quantum computer using classical computers.


Why is it important?

* Developing and bechmarking quantum computing applications and algorithms.
* Prototyping, verification, and debugging quantum programs.
* Studying the effect of noise.




### 3.2.1. Simulator backends - cuQuantum
CUDA Quantum's workhorse for quantum circuit simulation. It is a high performance library containing the following two types of simulators.

##### Statevector simulator    
* State vector simulators serve as the main vehicle for circuit simulations.
* They maintain an accurate representation of the quantum state, known as the state vector, throughout the simulation.
* Each gate that is applied corresponds to a matrix-vector multiplication.

##### Tensornet simulator
* The tensor network method is a technique that represents the quantum state of N qubits as a series of tensor contractions.
* The main challenge is to compute these tensor contractions efficiently.
* It can handle a large number of qubits for short circut depths.

Note: To run with the cutensornet target, you will need to pull the CUDA Quantum docker image with the tag latest-hpc.

#### 3.2.2 Density matrix simulator
* Simulates quantum circuits under the influence of noise.
* Currently, it calls the QPP library under the hood and has only CPU support.

To discuss the density matrix simulator further, we need to introduce a couple of new concepts.

### 4. Noise modeling

#### 4.1 Background

Noise refers to the multiple factors that can affect the accuracy of the calculations a quantum computer performs.

Common sources of errors in quantum computation.
- Quantum gate errors
- Measurement errors
- Decoherence
- Crosstalk

(Decoherence refers to the process by which a quantum system loses its quantum coherence, or its ability to exist in a superposition of states due to interaction with the environment.)
    
**Unitary gates vs Quantum channels**

When you model your quantum system as closed and look at transformations the whole system can go through, those are described by unitary evolutions. The evolution of a quantum system that interacts with its environment is described using quantum channels. The isolated quantum systems are an ideal case and practically, the systems have some degree of interaction with their surroundings.

**Wavefunction vs Density matrix**

The wavefunction or state vector gives a complete description of the quantum state of an isolated quantum system.
$|\psi\rangle$ --> ket notation for the quantum state represented as a vector. The density matrix representation is a more general representation that is used to describe noisy quantum evolution and decoherence. It can be used to describe the pure states as well as the mixed states, which are a statistical ensemble of the pure states.

In the density matrix notation, a pure state is given by

\begin{equation*}
\rho = |\psi\rangle |\psi\rangle.
\end{equation*}

A mixed state is repesentated as
    
\begin{equation*}
\rho = \sum_{j} p_j |\psi_j\rangle |\psi_j\rangle,
\end{equation*}
    where the coefficients $p_j$'s are the probabilities associated with each of the states in the ensemble.



##### Kraus Representation

The different sources of noise that we discussed above can be represnted mathematically using the Kraus operators.
    
\begin{equation*}
\rho \mapsto {\cal{N}}(\rho) = \sum_{j} K_j \rho K_j^{\dagger}
\end{equation*}

with the condition that
    
\begin{equation*}
\sum_{j} K_j K_j^{\dagger} = \mathbb{I}.
\end{equation*}



##### Some single-qubit errors

**Bit-flip error**

- The state of the qubit is chaged from |0⟩ to |1⟩ or vice-versa
- key-operator is Pauli X
- Kraus reprenetation

\begin{equation*}
    \rho = (1-p) \rho + p X\rho X
\end{equation*}
    with p in [0,1].


**Phase-flip error**

- The relative phase of a qubit is changed, but it's magnitude remains the same
- |0⟩ to -|0⟩ and |1⟩ to -|1⟩
- key-operator is Pauli Z

\begin{equation*}
    \rho = (1-p) \rho + p Z\rho Z
\end{equation*}
    with p in [0,1].    

**Amplitude damping**

- the qubit decayz from |1⟩ to the lower energy state |0⟩
    
\begin{equation*}
    \rho = K_1 \rho K_1^{\dagger} + K_2 \rho K_2^{\dagger},
\end{equation*}
 where $K_1 = [1,0;0,\sqrt{1-p}]$, $K_2 = [0,\sqrt{p};0,0]$ and p is the probability of decay.


#### 4.2 Noise modeling in CUDA Quantum with density-matrix simulator

Bit-flip channel

In [63]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports several different models of noise. In this case,
# we will examine the modeling of decoherence of the qubit state. This
# will occur from "bit flip" errors, wherein the qubit has a user-specified
# probability of undergoing an X-180 rotation.

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(1.0)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# Now we may define our simple kernel function and allocate a register
# of qubits to it.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.0`.
kernel.x(qubit)
# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# We should now see the qubit in the |1> state.
noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 0:1000 }

{ 1:1000 }



 Phase-flip channel
   

In [64]:
# Phase flip channel with `1.0` probability of the qubit
# undergoing a phase rotation of 180 degrees (π).
phase_flip = cudaq.PhaseFlipChannel(1.0)

Ampltiude damping channel

In [65]:
# Amplitude damping channel with `1.0` probability of the qubit
# decaying to the ground state.
amplitude_damping = cudaq.AmplitudeDampingChannel(1.0)
# We will apply this channel to any Hadamard gate on the qubit.
# Meaning, after each Hadamard on the qubit, there will be a
# probability of `1.0` that the qubit decays back to ground.
noise.add_channel('h', [0], amplitude_damping)

 Custom Noise Model

 Here, we demonstrate a custom noise model with the same Kraus operators as in the ampltiude damping channel, but following the same template we can build other noise models such as the Pauli noise model.

In [66]:
import cudaq
import numpy as np

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports custom noise models through the definition of
# `KrausChannel`'s. In this case, we will define a set of `KrausOperator`'s
# that  affect the same noise as the `AmplitudeDampingChannel`. This
# channel will model the energy dissipation within our system via
# environmental interactions. With a variable probability, it will
# return the qubit to the |0> state.

# We will begin by defining an empty noise model that we will add
# our Kraus Channel to.
noise = cudaq.NoiseModel()


# We will define our Kraus Operators within functions, as to
# allow for easy control over the noise probability.
def kraus_operators(probability):
    """See Nielsen, Chuang Chapter 8.3.5 for definition source."""
    kraus_0 = np.array([[1, 0], [0, np.sqrt(1 - probability)]],
                       dtype=np.complex128)
    kraus_1 = np.array([[0, 0], [np.sqrt(probability), 0]], dtype=np.complex128)
    return [kraus_0, kraus_1]


# Manually defined amplitude damping channel with `1.0` probability
# of the qubit decaying to the ground state.
amplitude_damping = cudaq.KrausChannel(kraus_operators(1.0))
# We will apply this channel to any Hadamard gate on the qubit.
# Meaning, after each Hadamard on the qubit, there will be a
# probability of `1.0` that the qubit decays back to ground.
noise.add_channel('h', [0], amplitude_damping)

# Now we may define our simple kernel function and allocate a qubit.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Then we apply a Hadamard gate to the qubit.
# This will bring it to `1/sqrt(2) (|0> + |1>)`, where it will remain
# with a probability of `1 - p = 0.0`.
kernel.h(qubit)

# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# The qubit will now have a 50/50 mix of measurements between
# |0> and |1>.
noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 0:1000 }

{ 0:461 1:539 }

